In [1]:
import pandas as pd
import datetime
import mariadb
import numpy as np
from pathlib import Path

In [2]:
csv_path = Path().cwd().parent.joinpath('voter_history_files/11-8-2016 - NOVEMBER 8, 2016 GENERAL:SPECIAL ELECTION.csv')
import_csv = pd.read_csv(csv_path, converters={'County Name': str, 'Voter Registration Number': str, 'Election Date':str, 'Election Type':str, 'Party': str, 'Ballot Style':str, 'Absentee':str, 'Provisional':str, 'Supplemental':str})

In [3]:
import_csv

,County Name,Voter Registration Number,Election Date,Election Type,Party,Ballot Style,Absentee,Provisional,Supplemental
0,APPLING,00004137,11/08/2016,GENERAL,,REGULAR,N,N,N
1,APPLING,00045353,11/08/2016,GENERAL,,REGULAR,N,N,N
2,APPLING,00047002,11/08/2016,GENERAL,,REGULAR,N,N,N
3,APPLING,00050679,11/08/2016,GENERAL,,REGULAR,N,N,N
4,APPLING,00122112,11/08/2016,GENERAL,,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...
4164405,DODGE,10037720,11/08/2016,GENERAL,,IN PERSON,Y,N,N
4164406,CHEROKEE,04251015,11/08/2016,GENERAL,,IN PERSON,Y,N,N
4164407,OCONEE,04034689,11/08/2016,GENERAL,,IN PERSON,Y,N,N
4164408,FRANKLIN,06415519,11/08/2016,GENERAL,,REGULAR,N,N,N


In [4]:
df = import_csv.copy()

In [5]:
df = df.rename(columns={'County Name': 'county_name', 'Voter Registration Number': 'voter_registration_number', 'Election Date': 'election_date', 'Election Type': 'election_type', 'Party': 'party', 'Ballot Style': 'ballot_style', 'Absentee': 'absentee', 'Provisional': 'provisional', 'Supplemental':'supplemental'})

In [6]:
def add_row_names(column):
    counter = 0
    row_name_list = []
    for row in column:
        counter += 1
        row_name_list.append(counter)
    return row_name_list

In [7]:
df.insert(0, 'row_names', add_row_names(df['county_name']))

In [8]:
def column_float_to_strings(column):
    string_list = []
    for entry in column:
        if str(entry) == 'nan':
            string_list.append("nan")
        else:
            string_list.append(str(entry))
    return string_list

In [9]:
def remove_last_two_digits(column):
    string_list = []
    for entry in column:
        string_list.append(entry[0:-2])
    return string_list

In [10]:
df['voter_registration_number'][df['voter_registration_number'].isnull()]

Series([], Name: voter_registration_number, dtype: object)

In [11]:
df

,row_names,county_name,voter_registration_number,election_date,election_type,party,ballot_style,absentee,provisional,supplemental
0,1,APPLING,00004137,11/08/2016,GENERAL,,REGULAR,N,N,N
1,2,APPLING,00045353,11/08/2016,GENERAL,,REGULAR,N,N,N
2,3,APPLING,00047002,11/08/2016,GENERAL,,REGULAR,N,N,N
3,4,APPLING,00050679,11/08/2016,GENERAL,,REGULAR,N,N,N
4,5,APPLING,00122112,11/08/2016,GENERAL,,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...,...
4164405,4164406,DODGE,10037720,11/08/2016,GENERAL,,IN PERSON,Y,N,N
4164406,4164407,CHEROKEE,04251015,11/08/2016,GENERAL,,IN PERSON,Y,N,N
4164407,4164408,OCONEE,04034689,11/08/2016,GENERAL,,IN PERSON,Y,N,N
4164408,4164409,FRANKLIN,06415519,11/08/2016,GENERAL,,REGULAR,N,N,N


In [12]:
def reformat_date(column):
    date_list = []
    for entry in column:
        date_list.append(f"{entry[6:10]}" + "-" + f"{entry[0:2]}" + "-" + f"{entry[3:5]}")
    return date_list

In [13]:
df['election_date'] = reformat_date(df['election_date'])

In [14]:
df

,row_names,county_name,voter_registration_number,election_date,election_type,party,ballot_style,absentee,provisional,supplemental
0,1,APPLING,00004137,2016-11-08,GENERAL,,REGULAR,N,N,N
1,2,APPLING,00045353,2016-11-08,GENERAL,,REGULAR,N,N,N
2,3,APPLING,00047002,2016-11-08,GENERAL,,REGULAR,N,N,N
3,4,APPLING,00050679,2016-11-08,GENERAL,,REGULAR,N,N,N
4,5,APPLING,00122112,2016-11-08,GENERAL,,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...,...
4164405,4164406,DODGE,10037720,2016-11-08,GENERAL,,IN PERSON,Y,N,N
4164406,4164407,CHEROKEE,04251015,2016-11-08,GENERAL,,IN PERSON,Y,N,N
4164407,4164408,OCONEE,04034689,2016-11-08,GENERAL,,IN PERSON,Y,N,N
4164408,4164409,FRANKLIN,06415519,2016-11-08,GENERAL,,REGULAR,N,N,N


In [15]:
output_path = Path().cwd().parent.joinpath('output_csv/general_2016.csv')
df.to_csv(output_path)

In [16]:
df.value_counts('election_type')

election_type
GENERAL    4164410
Name: count, dtype: int64